Retrieval pour la barre de recherche

In [89]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import re

In [90]:
# Initialiser le modèle SBERT et le Cross-Encoder
sbert_model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")
cross_encoder = CrossEncoder("cross-encoder/ms-marco-electra-base")
#"antoinelouis/crossencoder-electra-base-french-mmarcoFR"
#cross-encoder/ms-marco-MiniLM-L-12-v2
#cross-encoder/ms-marco-MiniLM-L-6-v2
#cross-encoder/camembert-base
#cross-encoder/ms-marco-electra-base

In [91]:
# Charger les données JSON
json_file_path = "linked_files.json"
with open(json_file_path, "r", encoding="utf-8") as file:
    json_data = json.load(file)

In [92]:
def extract_technique_section(content):
    # Rechercher la section Technique jusqu'au prochain titre de niveau 1
    match = re.search(r"# Technique\s*(.*?)(?=\n# |\Z)", content, re.DOTALL)
    if match:
        # Extraire le contenu
        technique_text = match.group(1).strip()
        # Supprimer toutes les lignes commençant par un '#' (titres et sous-titres)
        cleaned_text = re.sub(r"^#.*", "", technique_text, flags=re.MULTILINE)
        # Nettoyer les espaces et les lignes vides
        return "\n".join(line.strip() for line in cleaned_text.splitlines() if line.strip())
    return ""


In [ ]:
# Extraire les titres et le contenu technique
documents = []
for entry in json_data:
    title = entry["title"]
    content = entry["content"]
    # Extraire uniquement la partie "Technique" du contenu
    technique_section = extract_technique_section(content)
    # Combiner le titre et le contenu technique
    combined_text = f"{title} {technique_section}"
    #print(combined_text)
    documents.append(combined_text)

print(documents)

1070


In [95]:
# Calculer les embeddings des documents
embeddings = sbert_model.encode(documents)

# Indexer avec FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])
faiss.normalize_L2(embeddings)
index.add(embeddings)

In [109]:
# Fonction de recherche
def search(query, top_k):
    # Vectoriser la requêteJe 
    query_embedding = sbert_model.encode([query])
    faiss.normalize_L2(query_embedding)
    
    # Rechercher les documents les plus proches avec FAISS
    _, faiss_results = index.search(query_embedding, top_k)
    faiss_scores = [(documents[i], json_data[i]) for i in faiss_results[0]]
    
    # Réordonner les résultats avec le Cross-Encoder
    cross_scores = cross_encoder.predict([(query, doc[0]) for doc in faiss_scores])
    ranked_results = [x for _, x in sorted(zip(cross_scores, faiss_scores), reverse=True)]
    
    return ranked_results[:top_k]

In [113]:
import pandas as pd

def evaluate_search_performance(file_path, search_function, top_k=5, sample_size=789):  
    df = pd.read_excel(file_path)
    
    # Filtrer les lignes où les deux informations sont présentes
    valid_rows = df[(df['Solution/Action'].notna()) & (df['Code K'].notna())]
    print(len(valid_rows))
    # Sélectionner un échantillon aléatoire
    if len(valid_rows) > sample_size:
        sampled_rows = valid_rows.sample(n=sample_size, random_state=44)  # random_state pour la reproductibilité
    else:
        sampled_rows = valid_rows  # Utiliser toutes les lignes si moins de 200 sont disponibles
    
    # Initialiser les compteurs
    total_queries = 0
    correct_matches = 0
    
    # Parcourir les lignes sélectionnées
    for index, row in sampled_rows.iterrows():
        description = row['Solution/Action']
        code_k = row['Code K']
        
        total_queries += 1
        
        # Effectuer la recherche
        results = search_function(description, top_k)
        
        # Vérifier si le code K est dans les résultats
        found = False
        for result, metadata in results:
            if str(code_k) == metadata['identifier']:
                found = True
                break
        
        if found:
            correct_matches += 1
    
    # Calculer le score de précision
    if total_queries > 0:
        precision = correct_matches / total_queries
    else:
        precision = 0
    
    return precision


# Exemple d'utilisation
file_path = 'gain_CO2.xlsx'
precision_score = evaluate_search_performance(file_path, search)
print(f"Precision Score: {precision_score:.2f}")

789
Precision Score: 0.48


In [ ]:
# Exemple de recherche
query = "Comment isoler ma maison"
results = search(query)
for doc, metadata in results:
    print(f"Titre: {metadata['title']}")
    print(f"Code: {metadata['identifier']}")
    print("---")

Titre: Isolation des combles ou de toiture
Code: 250
---
Titre: Isolation des murs par l'intérieur
Code: 251
---
Titre: Isolation des toitures terrasses et couvertures de faible pente
Code: 255
---
Titre: Isolation des murs par l'extérieur
Code: 254
---
Titre: Isolation thermique par rideau d'air 
Code: 1620
---
Titre: Isolation d'un plancher
Code: 252
---
Titre: Isolation des fourreaux
Code: 917
---
Titre: Séparation des parties chauffées et non chauffées du bâtiment
Code: 266
---
Titre: Isolation des parois verticales entre les ateliers
Code: 269
---
Titre: Fermeture des rideaux et volets la nuit
Code: 767
---


Partie génération du chatbot avec web search

In [ ]:
import ollama
from ollama import chat
from ollama import ChatResponse
from duckduckgo_search import DDGS
from transformers import AutoTokenizer 
import nltk
from nltk.corpus import stopwords

In [ ]:
# Télécharger les stopwords si ce n'est pas déjà fait
# nltk.download('stopwords')
stop_words = set(stopwords.words('french'))

In [ ]:
# Fonction de Recherche Web avec DuckDuckGo
def web_search(query, num_results=5):
    """Effectue une recherche sur Internet et retourne les résultats."""
    # Filtrer les mots inutiles dans la question 
    filtered_query = filter_stop_words(query)
    
    results = []
    with DDGS() as ddgs:
        for r in ddgs.text(filtered_query, max_results=num_results):
            results.append(r["body"])
    return "\n".join(results)

# Fonction pour enlever les mots inutiles (stopwords)
def filter_stop_words(text):
    """Enlève les mots inutiles de la question."""
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

# Fonction de génération avec pipeline (Gemma:2b)
def generate_answer(prompt, retrieved_info, max_length=2000):
    """Génère une réponse via Gemma:2b."""
    response = chat(model='gemma:2b', messages=[
        {"role": "system", "content": f"""DOCUMENTS:\n{retrieved_info}\n\nQUESTION:\n{query}\n\nINSTRUCTIONS:
    Réponds à la QUESTION en utilisant exclusivement les DOCUMENTS fournis.
    Ta réponse doit être concise, claire et formulée avec tes propres mots.
    Donne des exemples concrets si possible.
    Donne des explications détaillées si nécessaire.
    N'hésite pas à donner plusieurs informations.
    Si la réponse n'est pas contenue dans les documents, réponds simplement que tu n'as pas trouvé d'information sur ce sujet.
    """},
        {"role": "user", "content": prompt}
    ])
    return response['message']['content']

# Système RAG : Recherche + Génération
def rag_system(question):
    """Système RAG qui combine recherche web et génération de réponse."""
    # Recherche Web pour obtenir des informations pertinentes
    retrieved_info = web_search(question)  
    
    # Créer le prompt avec la question et les infos récupérées
    prompt = f"Question: {question}\nInfos trouvées:\n{retrieved_info}\nRéponse:"
    
    # Génération de la réponse via Gemma:2b
    response = generate_answer(prompt, retrieved_info)
    
    return response

In [ ]:
if __name__ == "__main__":
    question = input("Pose ta question : ")
    response = rag_system(question)
    print("Réponse générée :", response)

Réponse générée : **Instructions pour isoler une maison**

1. Éliminez les ponts thermiques de la porte d'entrée.


2. Vérifiez que les porte et l'encadrement sont correctement ajustés.


3. Utilisez des isolants internes pour parer aux problèmes d'isolation.


4. Si nécessaire, construisez une maison en brique, bois ou béton.


5. Optez pour l'isolation extérieure de la maison neuve ou les travaux d'isolation des murs par l'intérieur.


Web Search restreinte à seulement des sites prédéfinis

In [ ]:
import requests
import ollama
from ollama import chat, ChatResponse
from transformers import AutoTokenizer

In [ ]:
# Clé API et ID moteur de recherche pour Google Custom Search
API_KEY = ''
SEARCH_ENGINE_ID = '14514bd4afbb647a0'

In [ ]:
# Fonction de Recherche Web avec Google Custom Search
def web_search(query, num_results=5):
    """Effectue une recherche sur Internet et retourne les résultats depuis des sites spécifiques."""

    # Filtrer les mots inutiles dans la question
    filtered_query = filter_stop_words(query)
    
    # Construire l'URL de l'API Google Custom Search
    url = f"https://www.googleapis.com/customsearch/v1?q={filtered_query}&cx={SEARCH_ENGINE_ID}&key={API_KEY}"
    
    response = requests.get(url)
    data = response.json()

    results = []
    if 'items' in data:
        for item in data['items'][:num_results]:
            results.append(item['snippet'])  # Extraire les extraits des résultats
    return "\n".join(results)

# Fonction pour enlever les mots inutiles (stopwords)
def filter_stop_words(text):
    """Enlève les mots inutiles de la question."""
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

# Fonction de génération avec pipeline (Gemma:2b)
def generate_answer(prompt, retrieved_info, max_length=500):
    """Génère une réponse via Gemma:2b."""
    response = chat(model='gemma:2b', messages=[
        {"role": "system", "content": f"""DOCUMENTS:\n{retrieved_info}\n\nQUESTION:\n{query}\n\nINSTRUCTIONS:
    Réponds à la QUESTION en utilisant exclusivement les DOCUMENTS fournis.
    Ta réponse doit être concise, claire et formulée avec tes propres mots.
    Donne des exemples concrets si possible.
    Donne des explications détaillées si nécessaire.
    N'hésite pas à donner plusieurs informations.
    Si la réponse n'est pas contenue dans les documents, réponds simplement que tu n'as pas trouvé d'information sur ce sujet.
    """},
        {"role": "user", "content": prompt}
    ])
    return response['message']['content']

# Système RAG : Recherche + Génération
def rag_system(question):
    """Système RAG qui combine recherche web et génération de réponse."""
    # Recherche Web pour obtenir des informations pertinentes
    retrieved_info = web_search(question)  
    
    # Créer le prompt avec la question et les infos récupérées
    prompt = f"Question: {question}\nInfos trouvées:\n{retrieved_info}\nRéponse:"
        
    # Génération de la réponse via Gemma:2b
    response = generate_answer(prompt, retrieved_info)
    
    return response

In [ ]:
# Exemple d'utilisation
question = "Comment isoler sa maison"
response = rag_system(question)
print(response)

**Comment isoler votre maison**

**1. Évaluer votre niveau de risque:**
* Déterminez la gravité des éléments de sécurité potentiels à l'intérieur de votre maison (chambres, portes, fenêtres, etc.).
* Évaluerz l'état de votre maison et les dommages éventuels.

**2. Planifiez et priorisez les travaux:**
* Priorisez les travaux les plus importants en fonction de la gravité des risques.
* Élaborez un plan détaillé pour chaque étape de l'isolation.

**3. Établir un plan d'urgence:**
* Prévoyez des moyens de communication et des systèmes de sécurité pendant qu'il y a une interruption d'électricité.
* Planifiez un plan de secours pour les situations d'urgence.

**4. Utilisez des protections:**
* Utilisez des protecteurs de soleil, des portes d'étanché et des fenêtres isolées pour protéger les personnes contre le soleil et les éléments.
* Utilisez des garde-fous et des systèmes de ventilation pour lutter contre les gaz toxiques.

**5. Privilégiez les mesures de sécurité:**
* Installez des port